In [ ]:
!pip install chatterbot

In [ ]:
!pip install playsound

In [ ]:
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer, ListTrainer

chatbot = ChatBot(name='Reachy',read_only = True, logic_adapters=[ {
            'import_path': 'chatterbot.logic.BestMatch',
                     'default_response': 'I am sorry, I do not understand. I am still learning. Please contact somebody for further assistance.',
                     'maximum_similarity_threshold': 0.90
        }
        ], trainer='chatterbot.trainers.ChatterBotCorpusTrainer')

# Create a new trainer for the chatbot
trainer = ChatterBotCorpusTrainer(chatbot)

# Train the chatbot based on the english corpus
trainer.train("chatterbot.corpus.english")

# Train the chatbot based on lists

small_talk = ['hi there!',
             'hi',
             'how do you do?',
             'how are you?',
             'i\'m cool.',
             'fine,you?',
             'always cool.',
             'i\m ok',
             'glad to heat that',
             'i feel awesome',
             'excellent, glad to hear that',
             'not so good',
             'sorry to hear that.',
             'what\'s your name?',
             'i\'m Reachy ']

specific = ['i want to go to bathroom',
            'ok, follow me to the bathroom',
            'i want to go to the kitchen',
            'ok, follow me to the kitchen',
            'i need to call Dan',
            'ok, please wait while I contact Dan',
            'i need to call Leah',
            'ok, please wait while I contact Leah']

list_trainer = ListTrainer(chatbot)

for item in (small_talk, specific):
    list_trainer.train(item)

In [ ]:
## For text to voice
from gtts import gTTS
import os
## For voice to text
import speech_recognition as sr
from playsound import playsound


In [ ]:
os.remove("speech0.mp3")
os.remove("speech1.mp3")

In [ ]:
def speak (text):
    global count

    language = 'en-us'
    speech = gTTS(text = text, lang = language, slow = False)
    sf = TemporaryFile()
    #speech.write_to_fp(sf) 
    filename = f'speech{count%2}.mp3'
    speech.save(filename)
    print( "Reachy: " + text)
    playsound(filename)
    os.remove(filename)
    count +=1
### Voice to text 

def listen ():
    #obtain audio from microphone 
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("[...listening...]")
        audio = r.listen(source)
    #Recognize speech using Sphinx

    try:
        listened = r.recognize_google(audio) 
        print("You said: " + listened)
    except sr.UnknownValueError:
        listened = "Not understand"
        print("could not understand audio")
    except sr.RequestError as e:
        listened = "Error"
        print("Error; {0}".format(e))
    except sr.WaitTimeoutError:
        listened = "Did not hear"
        print("WaitTimeoutError")
    
    return listened

In [ ]:
#### Keywords
key_words = ['kitchen',
            'bathroom',
            'call']

#### Chatbot structure    

#name = input('Enter Your Name: ')
welcome_text = "Welcome to Circuit Launch! I'm Reachy! What is your name?"# Service! Let me know how can I help you')
count = 0
speak(welcome_text)
name = listen()

salutation_text = "Hi "+ name +"! Let me know how can I help you"
speak(salutation_text)

while True:
    
    request = listen()
    request_words = request.split()
    
    for word in request_words:
        if word in key_words:
            command = word
            print("Keyword \""+ command +"\" detected. Executing command/action")
            
    if request=="Bye" or request=='bye' or request == "bye-bye":
        text = "Bye"
        speak(text)
        break
    else:
        response = chatbot.get_response(request)
        speak(response.text)
        